## Steps followed to create the django.
It takes as base the repository docker-django-example

- start the container: docker compose up
- get into the container: docker exec -it hellodjango-web-1 bash
- muestra como entro: python@6533204c6cff:/app/src$ 
- create the app: python manage.py startapp ecommerce
- it creates an app folder as ecommerce inside of the src

### Views.py
- Se crea un view. (ver django-views.ipynb)

### Urls
- Creamos un archivo urls.py si no existe

### Config/urls.py
- Actualizamos anexando el nuevo url

### Models.py
Inicializamos modelos de db
- Despues de agregar el modelo, hacemos la primera migracion con python manage.py makemigrations
- Despues aplicamos con python manage.py migrate

In [ ]:
python manage.py makemigrations
python manage.py migrate

### Config/Settings.py
Add the app to the config.

In [ ]:
INSTALLED_APPS = [
    "pages.apps.PagesConfig",
    "ecommerce.apps.EcommerceConfig", #<--- Added ecommerce app
]

The name was taken from already created apps.py

In [ ]:
from django.apps import AppConfig


class EcommerceConfig(AppConfig):
    name = 'ecommerce'


### ecommerce/admin.py
Add model to be applied

In [ ]:
from django.contrib import admin

# Register your models here.
from .models import ProductModel

admin.site.register(ProductModel)


### templates/ecommerce/list-view.html
Crear una carpeta de ecommerce denotr de src/pages/templates

### templates/ecommerce/product_detail.html
Crear template de detalles del product

Add the url en urls.py, including the designation of how the product_id will be received

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="product_model_list_view"),
    path("<int:product_id>/", views.product_model_detail_view, name="product_model_detail_view"), #<-- Added line
]


Agregar template product_detail.html

El product id will be passed as http://.../ecommerce/1
- Where 1 is the id

### templates/ecommerce/product_create.html
- Crea el template para crear productos en la base de datos.

- Agrega el view para crear productos

In [ ]:
def product_model_create_view(request: HttpRequest) -> HttpResponse:
    form = ProductModelForm(request.POST or None)
    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messages.success(request, "Produto creado con exito.")
        return HttpResponseRedirect("/ecommerce/{product_id}/".format(product_id=instance.id))
    template = "ecommerce/product_create.html"
    context = {"form": form}
    return render(request, template, context)

- Crea el form en ecommerce/forms.py

In [ ]:
from django import forms
from .models import ProductModel

class ProductModelForm(forms.ModelForm):
    class Meta:
        model = ProductModel
        fields = ['title', 'price']

- Crear mensajes: Usa The message framework in web
- https://docs.djangoproject.com/en/6.0/ref/contrib/message

{% if messages %}
<ul class="messages">
    {% for message in messages %}
    <li{% if message.tags %} class="{{ message.tags }}"{% endif %}>{{ message }}</li>
    {% endfor %}
</ul>
{% endif %}

Pega en un nuevo html en template llamado messages.html

- Update the urls.py

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="product_model_list_view"),
    path("<int:product_id>/", views.product_model_detail_view, name="product_model_detail_view"), 
    path("create/", views.product_model_create_view, name="product_model_create_view"), #<-- Added create view URL
]

### templates/ecommerce/product_update.html
Crear el template de update

In [ ]:
def product_model_update_view(request: HttpRequest, product_id: int) -> HttpResponse:
    instance = get_object_or_404(ProductModel, id=product_id)
    form = ProductModelForm(request.POST or None, instance=instance)
    if form.is_valid():
        instance = form.save(commit=False)
        instance.save()
        messages.success(request, "Produto actualizado con exito.")
        return HttpResponseRedirect("/ecommerce/{product_id}/".format(product_id=instance.id))
    template = "ecommerce/product_update.html"
    context = {"form": form}
    return render(request, template, context)

Agregar en urls

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="product_model_list_view"),
    path("<int:product_id>/", views.product_model_detail_view, name="product_model_detail_view"), 
    path("create/", views.product_model_create_view, name="product_model_create_view"), 
    path("<int:product_id>/update/", views.product_model_update_view, name="product_model_update_view"), #<-- Added update view URL
]


### templates/ecommerce/product_delete.html

Crear el template de delete

In [ ]:
def product_model_delete_view(request: HttpRequest, product_id: int) -> HttpResponse:
    instance = get_object_or_404(ProductModel, id=product_id)
    if request.method == "POST":
        instance.delete()
        messages.success(request, "Produto eliminado con exito.")
        return HttpResponseRedirect("/ecommerce/")
    template = "ecommerce/product_delete.html"
    context = {"product": instance}
    
    return render(request, template, context)

Agregar en urls

In [ ]:
from django.urls import path

from ecommerce import views

urlpatterns = [
    path("", views.product_model_list_view, name="product_model_list_view"),
    path("<int:product_id>/", views.product_model_detail_view, name="product_model_detail_view"), 
    path("create/", views.product_model_create_view, name="product_model_create_view"), 
    path("<int:product_id>/update/", views.product_model_update_view, name="product_model_update_view"),
    path("<int:product_id>/delete/", views.product_model_delete_view, name="product_model_delete_view"), #<-- Added delete view URL
]

### templates/ecommerce/search.html
- Update view.py

In [ ]:
@login_required(login_url='/admin/login/')
def product_model_list_view(request: HttpRequest) -> HttpResponse:
    query = request.GET.get("q", None)
    queryset = ProductModel.objects.all()
    if query is not None:
        queryset = queryset.filter(
            Q(title__icontains=query) |
            Q(description__icontains=query)
        )
    products = queryset
    template = "ecommerce/product_list.html"
    context = {"products": products}
    
    if request.user.is_authenticated:
        template = "ecommerce/product_list.html"
    else:
        template = "ecommerce/product_list_public.html"
    
    return render(request, template, context)

## Create super user


In [ ]:
python manage.py createsuperuser